In [1]:
# Preparing for work, using pandas, psycopg2 and warnings because I'm not using sqlalchemy
# and that produces warnings to use it

import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Preparing the connection to the database

conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [3]:
# Testing the connection

query_test = '''
    SELECT
        *
    FROM
        nyc_schools.high_school_directory
    LIMIT
        5
    ;
'''

df = pd.read_sql(query_test, conn)
df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [4]:
# Question 1: How many schools are there in each borough?

query_q1 = '''
    SELECT 
	    borough
	    ,COUNT(dbn)
    FROM
	    nyc_schools.high_school_directory
    GROUP BY
	    1
    ORDER BY
        1
    ;
'''

q1_answer = pd.read_sql(query_q1, conn)
q1_answer

,borough,count
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [5]:
# Question 2: What is the average % of English Language Learners (ELL) per borough?

query_q2 = '''
    SELECT
	    high_school_directory.borough
	    ,ROUND(AVG(school_demographics.ell_percent)::NUMERIC, 2) AS avg_ell_percent
    FROM
	    nyc_schools.high_school_directory
    LEFT JOIN
	    nyc_schools.school_demographics
    ON
	    high_school_directory.dbn = school_demographics.dbn
    GROUP BY 
        1
    ORDER BY
        1
    ;
'''

q2_answer = pd.read_sql(query_q2, conn)
q2_answer

,borough,avg_ell_percent
0,Bronx,NaN
1,Brooklyn,NaN
2,Manhattan,7.57
3,Queens,NaN
4,Staten Island,NaN


In [6]:
# Question 3: Using the data from the school demographics and high school directory,
# write a query to find the top 3 schools in each borough with the highest percentage
# of special education students (sped_percent)

query_q3 = '''
WITH temp_table AS (
SELECT
	nyc_schools.high_school_directory.borough
	,nyc_schools.high_school_directory.school_name
	,nyc_schools.school_demographics.sped_percent
	,DENSE_RANK() OVER (PARTITION BY nyc_schools.high_school_directory.borough ORDER BY nyc_schools.school_demographics.sped_percent DESC NULLS LAST) AS rank_num
FROM
	nyc_schools.high_school_directory
LEFT JOIN
	nyc_schools.school_demographics
ON
	nyc_schools.high_school_directory.dbn = nyc_schools.school_demographics.dbn
GROUP BY
	1, 2, 3
ORDER BY
	3
)

SELECT
	borough
	,school_name
	,sped_percent
FROM
	temp_table
WHERE
	rank_num <= 3 AND sped_percent IS NOT NULL
ORDER BY
	1, 3 DESC
;
'''

q3_answer = pd.read_sql(query_q3, conn)
q3_answer

,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


#### Conclusions:

- Among all boroughs, the Bronx has the most schools, and Staten Island the least.  
- The only available data for calculating the average percentage of English Language Learners is for Manhattan (7,57%); all other boroughs have NULL values.
- East Side Community School in Manhattan records the highest sped_percent in some years, outperforming other schools. Due to missing data from other boroughs, only Manhattan schools appear in the top 3 rankings.